In [1]:
import torch as tc
import numpy as np
import pickle
import os
from wish_utils import *
from WishDataSet import WishTubeFrameDataSet, get_transform, WishWorkspaceDataSet
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import utils
import threading
import multiprocessing as mp
from tqdm.notebook import tqdm

In [2]:
model_path = '/mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/tube_inc_range_withneighb_fasterrcnn_resnet50_fpn.pt'

In [3]:
device = tc.device('cuda') if tc.cuda.is_available() else tc.device('cpu')
device

device(type='cuda')

In [4]:
workspace_path = '/mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/Inferences/RealData/WISH00042730.npz'

In [5]:
ds = WishWorkspaceDataSet(workspace_path)

data set create for /mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/Inferences/RealData/WISH00042730.npz shape(1520, 128, 1540)


In [6]:
len(ds)

1520

In [7]:
x = ds[0]
x.shape, type(x), x.device

(torch.Size([3, 128, 1540]), torch.Tensor, device(type='cpu'))

In [8]:
data_loader = tc.utils.data.DataLoader(ds, batch_size=8, shuffle=False, num_workers=4)

In [9]:
len(data_loader)

190

In [10]:
tc.cuda.is_available() , tc.cuda.device_count()

(True, 1)

In [11]:
model = get_object_detection_model(num_classes=2)
model.load_state_dict(tc.load(model_path, map_location=device))
model = model.to(device)

/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
# ws = np.load(workspace_path)['arr_0']

In [13]:
# ws.shape

In [14]:
def plot_tube_detections(tube_idx, bin_data, predicted_boxes, score_threshold = 0.75):
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    cs = plt.contourf(bin_data)
    plt.colorbar(cs)
    
    plotted_predictions = 0
    for box, score in zip(predicted_boxes['boxes'], predicted_boxes['scores']):
        # print(f"prediction={box} score={score}")
        x_hat, y_hat = (box[0]+box[2])/2, (box[1]+box[3])/2
        if score < score_threshold:
            # print(f"----->Ignoring the prediction at x={x_hat} y={y_hat} with score={score}")
            plt.plot(x_hat, y_hat, marker="x", markersize=5, markeredgecolor="white", markerfacecolor="white")
            continue
            
        # print(f"Prediction coordinate x={x_hat} y={y_hat}")
        plt.plot(x_hat, y_hat, marker="x", markersize=5, markeredgecolor="red", markerfacecolor="red")
        plotted_predictions += 1

    plt.title(f"Tube-{tube_idx} Predicted-{plotted_predictions} peaks")
    plt.show()

In [15]:
# frame_to_tensor = A.Compose([ToTensorV2(p=1.0)])

In [16]:
# img = np.tile(frame_data[:, :, None], 3).astype(np.float32)

In [17]:
# b = a(image=img)

In [18]:
# b['image'].shape, type(b['image'])

In [19]:
# for tube in range(ws.shape[0]):
#     frame_data = ws[tube, ...]
#     img = np.tile(frame_data[:,:, None], 3).astype(np.float32)
#     img = frame_to_tensor(image=img)
#     # print(np.tile(frame_data[None,:,:], 3).shape)
#     # img_res = tc.from_numpy(img_res)

In [20]:
# img

In [21]:
# model.eval()

# probe = 0

# for img in range(ws.shape[0]):
#     frame_data = ws[tube, ...]
#     frame = np.tile(frame_data[:,:, None], 3).astype(np.float32)
#     frame = frame_to_tensor(image=frame)
#     with tc.no_grad():
#         prediction = model([frame['image'].to(device)])[0]
#         print(prediction)
#         nms_prediction = apply_nms(prediction, iou_thresh=0.001)
#         plot_tube_detections(frame['image'][0,:,:], nms_prediction, 0.50)
#         probe += 1
#         if probe > 1:
#             break

In [22]:
%%time
model.eval()
probe = 0
predicted_indices_with_score = []

for batch_idx, img_batch in enumerate(data_loader):
    for img_idx, img in enumerate(img_batch):
        tube_idx = batch_idx * data_loader.batch_size + img_idx
        with tc.no_grad():
            prediction = model([img.to(device)])[0]
            nms_prediction = apply_nms(prediction, iou_thresh=0.001)
            for box, score in zip(nms_prediction['boxes'], nms_prediction['scores']):
                tof = (box[0]+box[2])/2
                tube_res = (box[1]+box[3])/2
                predicted_indices_with_score.append([tube_idx, tube_res, tof, score])
            # plot_tube_detections(tube_idx, img[0,:,:], nms_prediction, 0.50)
            probe += 1
            
    # if probe > 1:
    #     break

CPU times: user 38.7 s, sys: 1.01 s, total: 39.7 s
Wall time: 40.1 s


In [23]:
len(predicted_indices_with_score)

688

In [24]:
pred_cpu = []
for pred in predicted_indices_with_score:
    pred_cpu.append([pred[0], pred[1].cpu(), pred[2].cpu(), pred[3].cpu()])

pred_cpu

[[11, tensor(54.7255), tensor(639.0803), tensor(0.6749)],
 [12, tensor(56.8230), tensor(657.9335), tensor(0.9700)],
 [12, tensor(56.2496), tensor(283.6342), tensor(0.2265)],
 [13, tensor(55.5720), tensor(668.3999), tensor(0.9499)],
 [14, tensor(58.5613), tensor(676.2388), tensor(0.8030)],
 [40, tensor(22.5045), tensor(298.5483), tensor(0.9868)],
 [50, tensor(88.5293), tensor(201.7269), tensor(0.1862)],
 [97, tensor(66.1760), tensor(428.8840), tensor(0.8724)],
 [98, tensor(67.0746), tensor(434.4890), tensor(0.9803)],
 [99, tensor(67.1554), tensor(440.7125), tensor(0.9288)],
 [127, tensor(79.1097), tensor(693.3325), tensor(0.9893)],
 [127, tensor(78.7824), tensor(323.7593), tensor(0.6711)],
 [128, tensor(79.7596), tensor(698.0956), tensor(0.9992)],
 [128, tensor(79.6710), tensor(328.0347), tensor(0.9741)],
 [129, tensor(78.6117), tensor(702.6392), tensor(0.9990)],
 [129, tensor(20.1676), tensor(546.7395), tensor(0.9455)],
 [130, tensor(21.8085), tensor(550.9728), tensor(0.9975)],
 [130, 

In [25]:
pred_cpu=np.array(pred_cpu)

In [26]:
len(pred_cpu[pred_cpu[:, -1] < 0.5])

155

In [28]:
np.savez_compressed('/mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/Inferences/RealData/WISH00042730_inferences_trained_inc_range_withneighb.npz', pred_cpu)

In [29]:
np.load('/mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/Inferences/RealData/WISH00042730_inferences_trained_inc_range_withneighb.npz')['arr_0']

array([[1.10000000e+01, 5.47255249e+01, 6.39080261e+02, 6.74949706e-01],
       [1.20000000e+01, 5.68230362e+01, 6.57933533e+02, 9.69980419e-01],
       [1.20000000e+01, 5.62495766e+01, 2.83634155e+02, 2.26467431e-01],
       ...,
       [1.42100000e+03, 6.65123596e+01, 3.36461151e+02, 3.01935196e-01],
       [1.45900000e+03, 7.00635529e+01, 4.44923889e+02, 9.18482900e-01],
       [1.46300000e+03, 2.58712616e+01, 1.96268982e+02, 3.77989799e-01]])

In [ ]:
arr = np.array(predicted_indices_with_score)
arr.shape

In [ ]:
len(arr[arr[:, -1]>=0.5])

In [ ]:
sdffsaf

### Data parallelism to speedup

In [ ]:
def doInference(img_batch):
    model.eval()
    predicted_indices_with_score = []
    batch_idx, img_batch = img_batch
    print(f"Starting {batch_idx}")
    for img_idx, img in enumerate(img_batch):
        tube_idx = batch_idx * batch_size + img_idx
        with tc.no_grad():
            prediction = model([img.to(device)])[0]
            nms_prediction = apply_nms(prediction, iou_thresh=0.001)
            for box, score in zip(nms_prediction['boxes'], nms_prediction['scores']):
                tof = (box[0]+box[2])/2
                tube_res = (box[1]+box[3])/2
                predicted_indices_with_score.append([tube_idx, tube_res, tof, score])
                # print(f"Inferred batch={batch_idx} tube={tube_idx}")
                
    print(f"Returning results for batch={batch_idx}...........")
    return predicted_indices_with_score

In [ ]:
%%time
data_loader = tc.utils.data.DataLoader(ds, batch_size=8, shuffle=False, num_workers=2)
batch_size = data_loader.batch_size
pool = mp.Pool(processes=2)

inferences = pool.map(doInference, enumerate(data_loader))

pool.close()
print("pool closed, waiting to join...")
pool.join()
print(len(inferences))

In [ ]:
things=[7,5,1,6,4,2]

def worker(thing):
    id, n = thing
    # time.sleep(n)
    print(f"id={id} thing={n} pid={os.getpid()}\n")
    return n
    
pool = mp.Pool(processes=mp.cpu_count())
results = pool.map(worker, enumerate(things))
pool.close()
pool.join()
for r in results:
    print(f"Result {r}") 